# Memory

In [1]:
!pip install langchain_community -q
!pip install langchain_openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.8 MB/s eta 0:00:00


In [2]:
from langchain_openai import ChatOpenAI

In [3]:
from google.colab import userdata
key = userdata.get('Openai')

In [4]:
model = ChatOpenAI(model = 'gpt-4o', api_key=key)

In [5]:
from langchain_core.messages import HumanMessage

In [6]:
model.invoke([HumanMessage(content='안녕 내이름은 정락이야')]).content

'안녕하세요, 정락님! 어떻게 도와드릴까요?'

In [7]:
model.invoke([HumanMessage(content='내 이름이 뭐야')]).content

'죄송하지만, 당신의 이름은 알 수 없습니다. 이름을 제공해 주시면 그에 맞춰 대화할 수 있습니다.'

In [8]:
from langchain_core.messages import AIMessage

In [9]:
model.invoke(
    [
        HumanMessage(content="안녕 내 이름은 정락이야"),
        AIMessage(content="안녕하세요, 정락님! 만나서 반갑습니다. 어떻게 도와드릴까요?"),
        HumanMessage(content="내 이름이 뭐야?"),
    ]
).content

'당신이 이전에 말씀해주신 이름은 정락입니다. 맞나요?'

## Message History

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [11]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
  if session_id not in store:
    store[session_id] = ChatMessageHistory()
  return store[session_id]

In [12]:
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [13]:
config = {'configurable': {'session_id':'session_1'}}

In [14]:
response = with_message_history.invoke(
    [HumanMessage(content="안녕 내 이름은 정락이야")],
    config=config,
)

response.content

'안녕하세요, 정락님! 만나서 반갑습니다. 오늘 어떻게 도와드릴까요?'

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐야?")],
    config=config,
)

response.content

'당신의 이름은 정락이라고 하셨습니다. 다른 도움이 필요하시면 말씀해 주세요!'

In [16]:
config = {"configurable": {"session_id": "session_2"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐야?")],
    config=config,
)

response.content

'죄송하지만, 저는 사용자의 이름을 알 수 없습니다. 어떻게 도와드릴까요?'

In [17]:
config = {"configurable": {"session_id": "session_1"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐야?")],
    config=config,
)

response.content

'당신의 이름은 정락이라고 하셨습니다. 다른 궁금한 점이 있으면 언제든지 말씀해 주세요!'

## Prompt templates 과 함께 쓰기

### chain 만들기

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [36]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 도움이 되는 AI Assistant이다. 모든 질문에 최선을 다해 답변하라.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [39]:
response = chain.invoke({"messages": [HumanMessage(content="안녕 난 구야")]})

response.content

'안녕하세요, 구야! 만나서 반가워요. 어떻게 도와드릴까요?'

In [25]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [27]:
config = {"configurable": {"session_id": "session_3"}}

response = with_message_history.invoke(
    [HumanMessage(content="안녕 난 훈이야")],
    config=config,
)

response.content

'안녕 훈이! 다시 만나서 반가워. 어떻게 지내? 또는 도움이 필요하면 뭐든지 말해줘!'

In [42]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐야?")],
    config=config,
)

response.content

'이전에 너 자신을 "훈이"라고 소개했어. 그게 네 이름이 맞니? 맞지 않다면, 너의 이름을 알려줘! 어떻게 도와줄까?'

In [43]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 도움이 되는 AI Assistant이다. 다음의 언어로 대답하라: {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [44]:
response = chain.invoke(
    {"messages": [HumanMessage(content="안녕 나는 곤이야")], "language": "English"}
)

response.content

'Hello, Gon! How can I assist you today?'

## RunnableWithMessageHistory 로 감싸기

In [45]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [47]:
config = {"configurable": {"session_id": "session_4"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="안녕 나는 곤이야")], "language": "Japanese"},
    config=config,
)

response.content

'こんにちは、ゴンさん。今日はどのようにお手伝いできますか？'

In [48]:
model.invoke(f'"{response.content}"가 한국어로 뭐야?').content

'한국어로는 "안녕하세요, 곤 씨. 오늘은 어떻게 도와드릴까요?"라고 번역할 수 있습니다.'

## Conversation History 관리하기

### chain 만들기

In [49]:
from langchain_core.runnables import RunnablePassthrough

In [50]:
def filter_messages(messages, k=10):
  msg = messages[-k:]
  print(f"len msgs : {len(msg)}")
  [print(f"{'ai' if type(msg) == AIMessage else 'human'}: {msg.content}")for msg in msg]
  return msg

In [51]:
chain = (RunnablePassthrough.assign(messages = lambda x: filter_messages(x['messages'])) | prompt | model)

In [52]:
messages = [
    HumanMessage(content="안녕하세요! 저는 구에요"),
    AIMessage(content="안녕하세요!"),
    HumanMessage(content="저는 바닐라 아이스크림을 좋아해요"),
    AIMessage(content="좋네요"),
    HumanMessage(content="2 + 2가 뭐죠?"),
    AIMessage(content="4에요"),
    HumanMessage(content="고마워요"),
    AIMessage(content="문제 없어요!"),
    HumanMessage(content="재미있어요?"),
    AIMessage(content="네, 재미있어요!"),
]

In [53]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="내이름이 뭐야?")],
        "language": "English",
    }
)
response.content  # 기억하지 못함

len msgs : 10
ai: 안녕하세요!
human: 저는 바닐라 아이스크림을 좋아해요
ai: 좋네요
human: 2 + 2가 뭐죠?
ai: 4에요
human: 고마워요
ai: 문제 없어요!
human: 재미있어요?
ai: 네, 재미있어요!
human: 내이름이 뭐야?


'죄송하지만, 저는 당신의 이름을 알 수 없습니다. 알려주시면 도움이 될 수 있을 것 같아요.'

In [54]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="내가 좋아하는 아이스크림은 뭐야?")],
        "language": "English",
    }
)
response.content  # 기억함

len msgs : 10
ai: 안녕하세요!
human: 저는 바닐라 아이스크림을 좋아해요
ai: 좋네요
human: 2 + 2가 뭐죠?
ai: 4에요
human: 고마워요
ai: 문제 없어요!
human: 재미있어요?
ai: 네, 재미있어요!
human: 내가 좋아하는 아이스크림은 뭐야?


'당신이 좋아하는 아이스크림은 바닐라 아이스크림이에요.'

## RunnableWithMessageHistory 로 감싸기

In [55]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [56]:
config = {"configurable": {"session_id": "session_5"}}

response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="내 이름이 뭐야?")],
        "language": "English",
    },
    config=config,
)

response.content

len msgs : 10
ai: 안녕하세요!
human: 저는 바닐라 아이스크림을 좋아해요
ai: 좋네요
human: 2 + 2가 뭐죠?
ai: 4에요
human: 고마워요
ai: 문제 없어요!
human: 재미있어요?
ai: 네, 재미있어요!
human: 내 이름이 뭐야?


'죄송하지만, 당신의 이름을 알 수는 없습니다. 알려주시면 기쁘게 기억하겠습니다!'

In [57]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="대화를 참고해서 내가 좋아하는 아이스크림은 뭐야?")],
        "language": "English",
    },
    config=config,
)

response.content

len msgs : 10
ai: 좋네요
human: 2 + 2가 뭐죠?
ai: 4에요
human: 고마워요
ai: 문제 없어요!
human: 재미있어요?
ai: 네, 재미있어요!
human: 내 이름이 뭐야?
ai: 죄송하지만, 당신의 이름을 알 수는 없습니다. 알려주시면 기쁘게 기억하겠습니다!
human: 대화를 참고해서 내가 좋아하는 아이스크림은 뭐야?


'죄송하지만, 이전 대화에서 당신이 좋아하는 아이스크림에 대한 정보는 없었습니다. 어떤 아이스크림을 좋아하는지 알려주시면 좋겠습니다!'

## Streaming

In [63]:
store.pop("session_6", None)

In [64]:
config = {"configurable": {"session_id": "session_6"}}

for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="안녕 재밌는 이야기하나 해줄래?")],
        "language": "Korean",
    },
    config=config,
):
    print(r.content, end="")

len msgs : 1
human: 안녕 재밌는 이야기하나 해줄래?
물론이죠! 다음은 재미있는 이야기입니다.

옛날 옛적, 작은 마을에 욕심 많은 구두쇠가 살고 있었습니다. 그의 이름은 영감이라 불렸고, 그는 사람들이 모르는 곳에 금화를 숨겨놓고 밤마다 그 금화를 세곤 했습니다. 어느 날, 영감은 자기가 숨겨놓은 금화가 너무 많다는 사실을 깨닫고, 좀 더 단단한 곳에 보관하기로 결심했습니다.

그래서 영감은 깊은 구덩이를 파고 그곳에 모든 금화를 묻었습니다. 매일 밤 그는 구덩이 옆에 앉아 자신의 비밀스러운 보물 상자를 보며 흡족해했습니다. 그런데 이 사실을 엿들은 도둑이 있었습니다. 도둑은 영감이 잠들기를 기다렸다가 모든 금화를 훔쳐 달아났습니다.

날이 밝고 영감은 평소처럼 보물을 확인하러 갔다가 금화가 모두 사라진 걸 발견했습니다. 그는 큰 소리로 울부짖으며 마을 사람들을 불러 모았습니다. "누군가 내 모든 금화를 훔쳐갔다!" 영감은 절규했습니다. 그때 현명한 마을 노인이 다가와 tranquil하게 말했습니다.

"영감님, 걱정하지 마세요. 여기에 큰 바위를 놓고 그것이 금화라고 생각하세요. 어차피 금화를 쓰지 않으셨으니, 이 바위도 똑같이 쓸 수 있지 않습니까?"

이 이야기는 우리가 가진 것의 진정한 가치를 생각해보는 계기를 제공합니다. 물질적인 것이 얼마나 중요한지보다는 그것을 어떻게 활용하는지가 더 중요하다는 교훈을 주죠.